# Example how to run the ZOEE-Optimization

This example uses the ZOEE EBM. Other models may be used but have to be inbound in ZOEE.modules.optimization first. The minimum requirements are described in this file, an the procedure how to execute it should follow the one shown here.

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from ZOEE.modules.configuration import importer, add_sellersparameters, parameterinterpolatorstepwise
from ZOEE.modules.variables import variable_importer, Vars
from ZOEE.modules.optimization import optimization, ZOEE_optimization
#from ZOEE import update_plotstyle, moving_average

#import matplotlib
#update_plotstyle()
#matplotlib.rcParams['lines.linewidth']=1.1

In [2]:
"""Import the target data"""
Lat_10,ZMT_10=np.loadtxt('ZOEE_transition/Data/ERA_ZMT_Mean_10deginterpolated.txt')
#Lat_2,ZMT_2=np.loadtxt('Data/ERA_1950_2deg.txt',delimiter=',')

In [3]:
"""Import the configuration that is required to run your specific model"""
config_base=importer('ZOEE_transition/Data/Sellers.ini')
parallel_config = {'number_of_parameters': 2, 'number_of_cycles': 1,
                                   'number_of_parallels': int(2 * 2 + 1)}
variable_importer(config_base,initialZMT=False,parallel=True,parallel_config=parallel_config)
config_base,ParasCTRL=add_sellersparameters(config_base,\
                                        parameterinterpolatorstepwise,'ZOEE_transition/Data/SellersParameterization.ini',2,0,True,True)


"""Decleration of optimization configuration"""
grid=Vars.Lat # grid specifies the distribution of cells over latitudes, provide in ONLY 1D
optimization_setup = optimization(mode='ZMT',
                            target=ZMT_10,
                            response=True, 
                            num_steps=10,
                            num_data=365*1000+1,
                            gamma0=1e-8,
                            cost_function_type='LeastSquare',
                            cost_ratio=1,
                            ZMT=ZMT_10,
                            GMT=288.15,
                            precision=0,
                            grid=grid)

"""Declaration of parameter setup, with initial parameters, parameter boundaries and 
the parameter pertubation to estimate the cost function gradient"""
P0=np.array([0.5,1.9e-15])
Pmin=np.array([0.45,1.425e-15])
Pmax=np.array([0.65,2.375e-15])
P_pert_ratio=1/10000
optimization_setup.give_parameters(P0,Pmin,Pmax,P_pert_ratio)
        
    
"""Import the class of your model that has to be defined in ZOEE.modules.optimization. And give it whatever 
configuration it requires"""
parameter_labels=[['func1','m'],['func1','gamma']]
parameter_levels=np.array([None,None])
elevation=-0.0065*np.array(ParasCTRL[1][1])
model = ZOEE_optimization(2,'ZMT',parameter_labels,parameter_levels,elevation,monthly=True)
#model_setup=[2,'ZMT',parameter_labels,parameter_levels,elevation,True]

"""Execture optimize to start the optimization, giving it your model imported in the step before and configuration
required to run your model"""
F, dF, P, Ptrans , gamma, data=optimization_setup.optimize(model,config_base)

Loading Configuration from: ZOEE_transition/Data/Sellers.ini
Loading Parameters from: ZOEE_transition/Data/SellersParameterization.ini


Iteration no.0
Starting controlrun with a temperature accuracy of 0.001 K on the GMT over 1000 datapoints.


/home/benni/.local/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)


KeyboardInterrupt: 